In [1]:
import torch
import torchvision
from models import LeNet5_CIFAR, LeNet5_MNIST, SimpleSNN, SimpleParaLif, LargerSNN , GeneralParaLIF, Frankenstein, LeNet5_Flexible, GeneralSNN
from scripts import train_model, test_model
from utils import load_data, get_object_name, is_leaky, plot_attack, printf
from attacks import foolbox_attack, art_attack
import foolbox as fb
from math import ceil
from art.attacks.evasion import SquareAttack, SimBA, BoundaryAttack, HopSkipJump, ZooAttack
import time
import pandas as pd
import matplotlib.pyplot as plt

original_device = torch.device('mps' if torch.backends.mps.is_available() else 'cuda' if torch.cuda.is_available() else 'cpu')

device = torch.device('mps' if torch.backends.mps.is_available() else 'cuda' if torch.cuda.is_available() else 'cpu')

##### Configuration #####

append_results_to_csv = True # setting this to False will replace the .csv

dataset = 'mnist'
plot_params = True

batch_size = 256
n_epochs = 1

num_steps = 20
tau_mem = 0.02
tau_syn = 0.02
spike_mode = 'SB'

models = [
    LeNet5_MNIST(),
    GeneralSNN(layer_sizes=(28*28, 2**9, 2**8, 2**7, 10), num_steps=num_steps),
    GeneralParaLIF(layer_sizes=(28*28, 2**9, 2**8, 2**7, 10), device=device, spike_mode=spike_mode, num_steps=num_steps, tau_mem=tau_mem, tau_syn=tau_syn)
]

noise_steps = torch.arange(4).long()

attack_functions = [
    art_attack,
    foolbox_attack,
    foolbox_attack,
    foolbox_attack,
]

attacks = [
    SquareAttack,
    fb.attacks.LinfFastGradientAttack(),
    fb.attacks.LinfDeepFoolAttack(),
    fb.attacks.LInfFMNAttack(),
]

epsilons = [0.01, 0.05, 0.1, 0.25, 0.5, 1]


##### Initialisation #####
'''
devices = [device if not is_leaky(m) else torch.device('cpu') for m in models]
models = [load_model(m, 'Baseline Models/models/' + n, d) for m, n, d in zip(models, model_filenames, devices)]
print('Models Loaded Successfully')
models = [m.to(d) for m, d in zip(models, devices)]
'''

optimizers = [
    torch.optim.Adam,
    torch.optim.SGD,
    torch.optim.Adamax,
]

learning_rates = [
    0.01,
    0.01,
    0.001
]

maximum_batches_to_run_attacks_on = ceil(1000 / batch_size) # can also be set manually


##### Initialisation #####

optimizers = [opt(m.parameters(), lr) for m, opt, lr in zip(models, optimizers, learning_rates)]
devices = [device if not is_leaky(m) else torch.device('cpu') for m in models]
models = [m.to(d) for m, d in zip(models, devices)]


##### Data #####

transforms = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0,0,0), (1,1,1)) if dataset in ['cifar', 'svhn'] else torchvision.transforms.Normalize(0, 1)
])

train_dataset, train_loader = load_data(dataset=dataset, path='data', train=True, batch_size=batch_size, transforms=transforms)
test_dataset, test_loader = load_data(dataset=dataset, path='data', train=False, batch_size=batch_size, transforms=transforms)


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

##### Train #####

all_results = {
    'model':[],
    'epochs_trained':[],
    'attack':[],
    'epsilon':[],
    'susceptibility_rate':[],
    'train_accuracy':[],
    'test_accuracy':[]
}

print('\n---------- Training ----------')
start_time = time.time()
for i, (model, optimizer, device) in enumerate(zip(models, optimizers, devices)):
    model_name = get_object_name(model)
    print(f'\nTraining model {i+1}: {model_name}')
    model, model_results = train_model(model, 
                                loader=train_loader, 
                                optimizer=optimizer,
                                n_epochs=n_epochs, 
                                device=device,
                                val_loader=test_loader)
    models[i] = model

    for attack, attack_function in zip(attacks, attack_functions):
        attack_name = get_object_name(attack)
        for epsilon in epsilons:
            total_successful_attacks, total_successful_classifications = 0, 0
            for batch, (images, labels) in enumerate(test_loader):
                printf(f'Attack: {attack_name} on model [{i+1}/{len(models)}]: {model_name}, epsilon: {epsilon}, batch: [{batch}/{maximum_batches_to_run_attacks_on}]')
                images, labels = images.to(original_device), labels.to(device)

                _, _, perturbed_prediction, original_prediction = attack_function(model,
                                                                                images,
                                                                                labels,
                                                                                attack,
                                                                                epsilon,
                                                                                device)
                perturbed_prediction = perturbed_prediction.to(device)
                original_prediction = original_prediction.to(device)
                correct_pre_attack = (original_prediction == labels)
                correct_post_attack = (perturbed_prediction == labels)
                n_successful_attacks = (correct_pre_attack & ~correct_post_attack)
                
                total_successful_attacks += n_successful_attacks.sum().to('cpu').item()
                total_successful_classifications += correct_pre_attack.sum().to('cpu').item()
                
                if batch == maximum_batches_to_run_attacks_on:
                    break
                
            all_results['model'] += [model_name]
            all_results['epochs_trained'] += [n_epochs]
            all_results['attack'] += [attack_name]
            all_results['epsilon'] += [epsilon]
            all_results['susceptibility_rate'] += [total_successful_attacks / total_successful_classifications]
            all_results['train_accuracy'] += [model_results['train accuracies'][-1]]
            all_results['test_accuracy'] += [model_results['val accuracies'][-1]]

print(f'\nTraining time: {time.time() - start_time:.1f} seconds.')




##### Save #####

all_results = pd.DataFrame(all_results)

path = 'Baseline Models/csvs/' + 'MNIST_attacks_by_epsilon_by_model.csv'

if not append_results_to_csv:
    while (ans := input('Are you sure you want to OVERWRITE the existing csv? y/n: ')) not in 'yn':
        continue
    if ans == 'n':
        append_results_to_csv = False


if append_results_to_csv:
    try:
        existing = pd.read_csv(path)
        final = pd.concat((existing, new))
        final.to_csv(path, index=False)
    except:
        print('No existing CSV found!\nCreating a new CSV.')
        all_results.to_csv(path, index=False)
else:
    all_results.to_csv(path, index=False)

print('\n\n---------- CSV SAVED ----------\n')

Attack: ABCMeta on model [1/3]: LeNet5_MNIST, epsilon: 0.01, batch: [0/4]<class 'generator'>
<class 'torch.Tensor'>


RuntimeError: Input type (MPSFloatType) and weight type (torch.FloatTensor) should be the same

In [ ]:

##### Plot #####

fig, axs = plt.subplots(1, 2)

for result in all_results:    
    axs[0].plot(result['epochs'], result['train accuracies'])
    axs[1].plot(result['epochs'], result['val accuracies'])

for i in [0, 1]:
    axs[i].set_xlabel('Epochs')
    axs[i].set_ylabel('Accuracy')
    axs[i].legend([get_object_name(m, neat=True) for m in models], loc = 'lower right')
    
axs[0].set_title('Train Accuracies')
axs[1].set_title('Test Accuracies')

if plot_params:
    fig.suptitle(
        f'Models: {[get_object_name(m, neat=True) for m in models]}\n' +
        f'Optims: {[get_object_name(o, neat=True) for o in optimizers]}\n' +
        f'lrs: {[lr for lr in learning_rates]}\n' +
        f'Dataset: {dataset.upper()}'
    )
else:
    fig.suptitle(
        f'Dataset: {dataset.upper()}'
    )

plt.show()